In [1]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib
from scipy import signal
import random

import h5py
from IPython.utils import io

from joblib import Parallel, delayed
from tqdm import tqdm
import sys

In [2]:
%matplotlib widget

%reload_ext autoreload
%autoreload 2

In [3]:
sys.path.append('../../../../nadavp/new_trap/Tools/')
sys.path.append('../../../../nadavp/new_trap/Tools/StatFramework/')
# sys.path.append('../../../../nadavp/opt_lev_analysis/lib/')
import BeadDataFile
from discharge_tools import *

In [4]:
from likelihood_calculator import likelihood_analyser
from likelihood_calculator import GravityFramework
from likelihood_calculator import auxiliary_functions as aux
gfw = GravityFramework.GravityFramework()

In [5]:
# import bead_util as bu
# import calib_util as cal
# import transfer_func_util as tf
# import configuration as config

### Discharge data

In [59]:
ll = likelihood_analyser.LikelihoodAnalyser()
bandwidth = 2
decimate = 10
drive_freq = 71
fsamp = 5000
def func1_scale(i):
    bb = step_cal_files[i]
        
    freq2 = np.fft.rfftfreq(len(bb.electrode_data[0]), d=1./5000)
    fft_angles = np.angle(np.fft.rfft(bb.electrode_data[0]))
    phi_tmp2 = (fft_angles[freq2==drive_freq]+np.pi/2)%(2*np.pi)
    print(phi_tmp2,np.std(bb.electrode_data[0])*np.sqrt(2))

    fit_kwargs = {'A': 0, 'f': drive_freq, 'phi': phi_tmp2, 
                  'error_A': 1, 'error_f': 1, 'error_phi': 0.5, 'errordef': 1,
                  'limit_phi': [-2 * np.pi, 2 * np.pi], 
                  'limit_A': [-1, 1], 
                  'print_level': 0, 'fix_f': True, 'fix_phi': True}

    frequency = fit_kwargs['f']

    b, a = signal.butter(3, [2.*(drive_freq-bandwidth/2.)/fsamp, 2.*(drive_freq+bandwidth/2.)/fsamp ], btype = 'bandpass')
    xx2 = signal.filtfilt(b, a, bb.x2)[::decimate]
    
    m1_tmp = ll.find_mle_sin(xx2, fsamp=5000/decimate, noise_rms=1, plot=False, suppress_print=True, **fit_kwargs)
    
    return m1_tmp.values[0]

In [60]:
bb = step_cal_files[0]

In [61]:
func1_scale(0)

[0.08190331] 9.996283880412916


0.00013055475031054087

In [7]:
step_cal_dir = r'/data/new_trap/20210622/bead1/Discharge/FineDischarge/'
step_cal_files = load_dir(step_cal_dir, start_file=300)

580  files in folder
280  files loaded


In [62]:
amps3 = []
for i in range(len(step_cal_files)):
    print(i,'/',len(step_cal_files))
    amps3.append(func1_scale(i))

0 / 280
[0.08190331] 9.996283880412916
1 / 280
[0.08190909] 9.996281304225382
2 / 280
[0.08189878] 9.996281537733502
3 / 280
[0.08188683] 9.996288002004473
4 / 280
[0.08188301] 9.996283855239406
5 / 280
[0.0818812] 9.996280896041402
6 / 280
[0.08188132] 9.996280781109695
7 / 280
[0.08187602] 9.996281466488083
8 / 280
[0.08187166] 9.996283836062767
9 / 280
[0.08186824] 9.99628049649982
10 / 280
[0.08185259] 9.99628122742368
11 / 280
[0.08184229] 9.996278292760515
12 / 280
[0.08184375] 9.9962872679154
13 / 280
[0.08185011] 9.996282311106809
14 / 280
[0.08185688] 9.996282232848417
15 / 280
[0.08187448] 9.99628608262862
16 / 280
[0.0818733] 9.996285608676523
17 / 280
[0.08187297] 9.996284865507036
18 / 280
[0.08186803] 9.996282915362116
19 / 280
[0.08185791] 9.996282562713075
20 / 280
[0.08185112] 9.996280282430336
21 / 280
[0.08186028] 9.996284879184568
22 / 280
[0.08187175] 9.99628085579499
23 / 280
[0.08186014] 9.996283020975962
24 / 280
[0.08186719] 9.996287090219038
25 / 280
[0.081872

[0.08190371] 9.996283979268693
215 / 280
[0.08190964] 9.996278217408733
216 / 280
[0.08190812] 9.996281835959412
217 / 280
[0.08189192] 9.996278896802057
218 / 280
[0.08189368] 9.996275926505898
219 / 280
[0.08190671] 9.996279128442351
220 / 280
[0.08179691] 9.99627468897426
221 / 280
[0.08182042] 9.99627516703645
222 / 280
[0.08180523] 9.99627521228304
223 / 280
[0.08181146] 9.99627824130788
224 / 280
[0.08183009] 9.996278267050043
225 / 280
[0.08180829] 9.996275926682001
226 / 280
[0.08182235] 9.996279022607434
227 / 280
[0.08182145] 9.996281920292775
228 / 280
[0.08182219] 9.99628283021442
229 / 280
[0.08184658] 9.996279959458805
230 / 280
[0.08186107] 9.996278940572699
231 / 280
[0.08185479] 9.996280217483479
232 / 280
[0.0818335] 9.996273964556677
233 / 280
[0.08184075] 9.99627808392072
234 / 280
[0.08184671] 9.996280048447549
235 / 280
[0.0818646] 9.996273892287004
236 / 280
[0.08186131] 9.99627225841259
237 / 280
[0.08186464] 9.996274749993654
238 / 280
[0.08185296] 9.9962793456

In [90]:
# amps = np.array(amps)
amps3 = np.array(amps3)
_,ax = plt.subplots(figsize=(9.5,4))
# scale = 326.26

# Perrinf
ax.scatter(range(len(amps3)), amps3/scale, label='fit phase')
# ax.scatter(range(len(amps3)), amps3/scale, label='fit fixed-phases')

ax.set(xlabel='dataset', ylabel='Electron charges', ylim=(-1.2,6.5), xlim=(160,None))
ax.grid(axis='y')
ax.minorticks_on()
ax.grid(which='major', linestyle='-', linewidth='0.5', color='red')
ax.grid(which='minor', linestyle=':', linewidth='0.5', color='black')
# ax.legend()
from matplotlib.ticker import MaxNLocator
ax.yaxis.set_major_locator(MaxNLocator(integer=True))

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [75]:
scale = np.abs(np.mean(amps3[180:200]))
print("Scale is ", scale)
bb = step_cal_files[-1]
voltage = np.std(bb.electrode_data[0])*np.sqrt(2)
print("voltage: ", voltage)

Scale is  3.373597605932366e-05
voltage:  9.996282495738035


In [97]:
ll = likelihood_analyser.LikelihoodAnalyser()
bandwidth = 2
decimate = 10
drive_freq = 71
phase_calib = 0.081

def get_amp(i):
    
    fname = r'/data/new_trap/20210622/bead1/Neutrality/Long_X_both_10V_71Hz_height_m2_files_2000/Discharge_'+str(i)+'.h5'
    bb = BeadDataFile.BeadDataFile(fname=fname)


    fit_kwargs = {'A': 0, 'f': drive_freq, 'phi': phase_calib, 
                  'error_A': 1, 'error_f': 1, 'error_phi': 0.5, 'errordef': 1,
                  'limit_phi': [-2 * np.pi, 2 * np.pi], 
                  'limit_A': [-1, 1], 
                  'print_level': 0, 'fix_f': True, 'fix_phi': True}
    
    b, a = signal.butter(3, [2.*(drive_freq-bandwidth/2.)/fsamp, 2.*(drive_freq+bandwidth/2.)/fsamp ], btype = 'bandpass')
    xx2 = signal.filtfilt(b, a, bb.x2)[::decimate]
    
    m1_tmp = ll.find_mle_sin(xx2, fsamp=5000/decimate, noise_rms=1, plot=False, suppress_print=True, **fit_kwargs)
    
    return m1_tmp.values[0]

In [98]:
i=0
get_amp(i)/scale

0.043623305086996494

In [86]:
i = 0
fname = r'/data/new_trap/20210622/bead1/Neutrality/Short_X_both_10V_71Hz_height_m2_files_500/Discharge_'+str(i)+'.h5'
bb = BeadDataFile.BeadDataFile(fname=fname)
voltage2 = np.std(bb.electrode_data[0])*np.sqrt(2)
bias2 = np.mean(bb.electrode_data[0])
print("voltage, bias: ", voltage2, bias2)

voltage, bias:  9.99627842831243 -0.003219391483040524


In [99]:
amps_w_fb_no_spin_x_10V_height_m2_long = []
for i in range(2000):
    print(i,'/500')
    try:
        amps_w_fb_no_spin_x_10V_height_m2_long.append(get_amp(i))
    except:
        print("******")
amps_w_fb_no_spin_x_10V_height_m2_long = np.array(amps_w_fb_no_spin_x_10V_height_m2_long)

0 /500
1 /500
2 /500
3 /500
4 /500
5 /500
6 /500
7 /500
8 /500
9 /500
10 /500
11 /500
12 /500
13 /500
14 /500
15 /500
16 /500
17 /500
18 /500
19 /500
20 /500
21 /500
22 /500
23 /500
24 /500
25 /500
26 /500
27 /500
28 /500
29 /500
30 /500
31 /500
32 /500
33 /500
34 /500
35 /500
36 /500
37 /500
38 /500
39 /500
40 /500
41 /500
42 /500
43 /500
44 /500
45 /500
46 /500
47 /500
48 /500
49 /500
50 /500
51 /500
52 /500
53 /500
54 /500
55 /500
56 /500
57 /500
58 /500
59 /500
60 /500
61 /500
62 /500
63 /500
64 /500
65 /500
66 /500
67 /500
68 /500
69 /500
70 /500
71 /500
72 /500
73 /500
74 /500
75 /500
76 /500
77 /500
78 /500
79 /500
80 /500
81 /500
82 /500
83 /500
84 /500
85 /500
86 /500
87 /500
88 /500
89 /500
90 /500
91 /500
92 /500
93 /500
94 /500
95 /500
96 /500
97 /500
98 /500
99 /500
100 /500
101 /500
102 /500
103 /500
104 /500
105 /500
106 /500
107 /500
108 /500
109 /500
110 /500
111 /500
112 /500
113 /500
114 /500
115 /500
116 /500
117 /500
118 /500
119 /500
120 /500
121 /500
122 /500
123

926 /500
927 /500
928 /500
929 /500
930 /500
931 /500
932 /500
933 /500
934 /500
935 /500
936 /500
937 /500
938 /500
939 /500
940 /500
941 /500
942 /500
943 /500
944 /500
945 /500
946 /500
947 /500
948 /500
949 /500
950 /500
951 /500
952 /500
953 /500
954 /500
955 /500
956 /500
957 /500
958 /500
959 /500
960 /500
961 /500
962 /500
963 /500
964 /500
965 /500
966 /500
967 /500
968 /500
969 /500
970 /500
971 /500
972 /500
973 /500
974 /500
975 /500
976 /500
977 /500
978 /500
979 /500
980 /500
981 /500
982 /500
983 /500
984 /500
985 /500
986 /500
987 /500
988 /500
989 /500
990 /500
991 /500
992 /500
993 /500
994 /500
995 /500
996 /500
997 /500
998 /500
999 /500
1000 /500
1001 /500
1002 /500
1003 /500
1004 /500
1005 /500
1006 /500
1007 /500
1008 /500
1009 /500
1010 /500
1011 /500
1012 /500
1013 /500
1014 /500
1015 /500
1016 /500
1017 /500
1018 /500
1019 /500
1020 /500
1021 /500
1022 /500
1023 /500
1024 /500
1025 /500
1026 /500
1027 /500
1028 /500
1029 /500
1030 /500
1031 /500
1032 /500
1033

1755 /500
1756 /500
1757 /500
1758 /500
1759 /500
1760 /500
1761 /500
1762 /500
1763 /500
1764 /500
1765 /500
1766 /500
1767 /500
1768 /500
1769 /500
1770 /500
1771 /500
1772 /500
1773 /500
1774 /500
1775 /500
1776 /500
1777 /500
1778 /500
1779 /500
1780 /500
1781 /500
1782 /500
1783 /500
1784 /500
1785 /500
1786 /500
1787 /500
1788 /500
1789 /500
1790 /500
1791 /500
1792 /500
1793 /500
1794 /500
1795 /500
1796 /500
1797 /500
1798 /500
1799 /500
1800 /500
1801 /500
1802 /500
1803 /500
1804 /500
1805 /500
1806 /500
1807 /500
1808 /500
1809 /500
1810 /500
1811 /500
1812 /500
1813 /500
1814 /500
1815 /500
1816 /500
1817 /500
1818 /500
1819 /500
1820 /500
1821 /500
1822 /500
1823 /500
1824 /500
1825 /500
1826 /500
1827 /500
1828 /500
1829 /500
1830 /500
1831 /500
1832 /500
1833 /500
1834 /500
1835 /500
1836 /500
1837 /500
1838 /500
1839 /500
1840 /500
1841 /500
1842 /500
1843 /500
1844 /500
1845 /500
1846 /500
1847 /500
1848 /500
1849 /500
1850 /500
1851 /500
1852 /500
1853 /500
1854 /500


In [100]:
data = amps_w_fb_no_spin_x_10V_height_m2/scale*voltage/voltage2
data2 = amps_w_fb_no_spin_x_10V_height_m2_long/scale*voltage/voltage2
# data3 = amps_w_fb_spin_xz_y_100V_100V_sweep3_m37_5_height_0_4/scale*voltage/voltage1
_, ax= plt.subplots(figsize=(9.5,4))
ax.scatter(range(len(data)), data)
ax.scatter(np.arange(len(data2)), data2)
# ax.scatter(np.arange(len(data3)), data3)
ax.plot(np.arange(0,2000), np.arange(0,2000)*0, '--r')
ax.set(xlabel='Dataset', ylabel=r'$\epsilon$')
# ax.legend()
print(np.mean(data2), np.std(data2)/np.sqrt(len(data2)))

/home/analysis_user/.local/lib/python3.6/site-packages/ipykernel_launcher.py:4: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  after removing the cwd from sys.path.


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

0.003340942583312365 0.0014578265042021895


In [104]:
_,ax = plt.subplots()
ax.hist(data2, bins=50)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

(array([  2.,   0.,   0.,   0.,   2.,   2.,   4.,   5.,   7.,   5.,  12.,
         16.,  18.,  30.,  38.,  43.,  47.,  65.,  80., 108.,  93., 111.,
        109., 111., 109., 118., 116.,  93., 104.,  94.,  80.,  78.,  51.,
         57.,  31.,  36.,  31.,  24.,  21.,  12.,  14.,   2.,   5.,   3.,
          4.,   3.,   1.,   1.,   2.,   2.]),
 array([-0.23023105, -0.22087951, -0.21152798, -0.20217644, -0.19282491,
        -0.18347337, -0.17412184, -0.1647703 , -0.15541876, -0.14606723,
        -0.13671569, -0.12736416, -0.11801262, -0.10866109, -0.09930955,
        -0.08995802, -0.08060648, -0.07125495, -0.06190341, -0.05255187,
        -0.04320034, -0.0338488 , -0.02449727, -0.01514573, -0.0057942 ,
         0.00355734,  0.01290887,  0.02226041,  0.03161194,  0.04096348,
         0.05031501,  0.05966655,  0.06901809,  0.07836962,  0.08772116,
         0.09707269,  0.10642423,  0.11577576,  0.1251273 ,  0.13447883,
         0.14383037,  0.1531819 ,  0.16253344,  0.17188498,  0.18123651,
 

### Fit the data

In [298]:
x1 = np.arange(100)
x2 = np.arange(1000)
## estimate sensitivity from identical dataets
z,cov = np.polyfit(x1, data, 1, cov=True)
p1 = np.poly1d(z)
z2, cov2 = np.polyfit(x1, data2, 1, cov=True)
p2 = np.poly1d(z2)
# z3, cov3 = np.polyfit(x1, data3, 1, cov=True)
# p3 = np.poly1d(z3)

In [299]:
cov, cov2, cov3

(array([[  1.62594735e-10,  -8.04843939e-09],
        [ -8.04843939e-09,   5.33879813e-07]]),
 array([[  1.86933225e-10,  -9.25319462e-09],
        [ -9.25319462e-09,   6.13795243e-07]]),
 array([[  2.31818913e-12,  -4.62478732e-10],
        [ -4.62478732e-10,   1.23173502e-07]]))

In [300]:
x = -z2[1]; dx = np.sqrt(cov2[1,1])
y = z2[0]; dy = np.sqrt(cov2[0,0])
dz = np.abs(x/y)*np.sqrt((dx/x)**2+(dy/y)**2)
print(dz)
print(-z[1]/z[0], -z2[1]/z2[0], -z3[1]/z3[0])
print(z[0], z2[0], z3[0])
print(p1(-z2[1]/z2[0]), p1(-z2[1]/z2[0]+dz), p1(-z2[1]/z2[0]-dz))

0.99042769671
44.4536117505 15.1590864857 72.9563426674
-0.000617756631611 0.000818234826546 -0.000144139307458
0.0180968872522 0.0174850439744 0.01870873053


In [314]:
-(z2[1]+z[1])/(z2[0]+z[0])*2/100, 1.5*50/2

(-1.5021903964710268, 37.5)

In [170]:
_, ax= plt.subplots(figsize=(9.5,4))
# ax.scatter(np.arange(len(data)), data, label='-50V to 50V bias sweep - 100V, xz-spin, w/ height fb')
# ax.scatter(np.arange(len(data2)), data2, label='-50V to 50V bias sweep - 100V, xz-spin, w/ height fb')
# ax.scatter(np.arange(len(data3)), data3/scale*scale2, label='-50V to 50V bias sweep - 75V-125V, xz-spin, w/ height fb')
ax.plot(np.arange(0,500), np.arange(0,500)*0, '--r')
ax.set(xlabel='Dataset', ylabel=r'$\epsilon$')
ax.plot(x1, p1(x1), 'b--', label='fit1')
ax.plot(x1, p2(x1), 'r--', label='fit2')
ax.plot(x1, p3(x1), 'r--', label='fit3')
ax.legend()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …